Основная идея решения заключается в том, что последовательность данных является очень важным признаком. Те, если внимательно посмотреть на данные, можно обнаружить много закономерностей, для слов, идущих по порядку. Получается, что для получения максимального скора необходимо построить модель, которая не просто хорошо определяет, является ли слово "Иванов" фамилией, а которое определяет это, при условии, что перед этим словом было слово "ИВАНОВ", а после еще 5 похожих на него слов. 

Кому лень смотреть код (а он абсолютно не стоит смотрения), основные фичи: написание (с большой буквы, с маленькой, капсом), то же самое для предыдущего слова, является ли предыдущее слово дублем, является ли предыдущее слово "однокоренным" (начинается ли текущее слово с предыдущего без трех букв), агрегация сколько в группе подряд идущих дублей, подряд идущих похожих слов, плюс OHE на окончания (последние две буквы). на второй неделе добавились признаки из pymorphy - падеж, часть речи, множественное число для текущего слова и для предыдущего, natasha и pymystem3, потыренные у Кирилла, признаки про второе предыдущее слово. (как позже выяснилось, последние два не привели к улучшению скора на приватном лб)

Все это отправлялось в XGBClassifier, валидировалась на отложенной выборке, данные делила по букве, чтобы не нарушать последоватьельность похожих слов.  

в конце постобработка - для слов, которые являются дублями учитывая регистр, первому присваивается 0, второму 1. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from scipy.sparse import hstack

In [2]:
#читаем данные
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [6]:
#добавляем признак - слово написанное строчными буквами
train['word']=train['Word'].apply(lambda x: x.lower())
test['word']=test['Word'].apply(lambda x: x.lower())

In [4]:
#признак написание слова: капсом, с заглавной буквы, с маленькой буквы
def is_starts(x):
    if x.isupper(): r=2
    elif x[0].isupper(): r=1
    elif x[0].islower(): r=0
    else: r=100
    return r

train['is_starts']=train['Word'].map(is_starts)
test['is_starts']=test['Word'].map(is_starts)

In [5]:
#Признаки связанные с предыдущим словом. Какое написание, является ли дублем, 
#является ли "однокоренным" словом (текущее слово начинается с предыдущего слова без трех последних букв )

train['nb_is_starts']=np.append(np.array([0]), train['is_starts'][:-1].values)

nb=train.word[0]
is_dbl=[]
is_syn=[]
for i in train.word[1:]:
    if nb==i:
        is_dbl.append(1)
        is_syn.append(1)
    else:
        is_dbl.append(0)
        if (len(nb)>3 and i.startswith(nb[:-3])) or (len(nb)==3 and i.startswith(nb[:-2]) or i.startswith(nb)):
            is_syn.append(1)
        else: is_syn.append(0)
    nb=i
train['is_dbl']=np.append(np.array([0]), np.array(is_dbl))
train['is_syn']=np.append(np.array([0]), np.array(is_syn))

#test

test['nb_is_starts']=np.append(np.array([0]), test['is_starts'][:-1].values)

nb=test.word[0]
is_dbl=[]
is_syn=[]
for i in test.word[1:]:
    if nb==i:
        is_dbl.append(1)
        is_syn.append(1)
    else:
        is_dbl.append(0)
        if (len(nb)>3 and i.startswith(nb[:-3])) or (len(nb)==3 and i.startswith(nb[:-2]) or i.startswith(nb)):
            is_syn.append(1)
        else: is_syn.append(0)
    nb=i
test['is_dbl']=np.append(np.array([0]), np.array(is_dbl))
test['is_syn']=np.append(np.array([0]), np.array(is_syn))


In [6]:
#теперь добавим аггригированные значения. сколько синонимов в группе, сколько дублей у слова.
l=[]
n=0
for i in train.is_syn:
    if i==0: 
        l=np.append(l, [n]*n+[0])
        n=0
    else:
        n+=1

train['agg_syn']=np.append(l, [n]*n).astype(int)

train['agg_dbl']=train.groupby('word')['Word'].transform('count')

#test
l=[]
n=0
for i in test.is_syn:
    if i==0: 
        l=np.append(l, [n]*n+[0])
        n=0
    else:
        n+=1
       
test['agg_syn']=np.append(l, [n]*n).astype(int)

test['agg_dbl']=test.groupby('word')['Word'].transform('count')

In [7]:
#признаки состоит ли слово только из букв и начинается ли с "о'"
train['is_a']=train['Word'].apply(lambda x: x.isalpha()).astype(int)
train['apst']=train['word'].apply(lambda x: 1 if x.startswith("о'") else 0).astype(int)


test['is_a']=test['Word'].apply(lambda x: x.isalpha()).astype(int)
test['apst']=test['word'].apply(lambda x: 1 if x.startswith("о'") else 0).astype(int)

In [8]:
#работа с окончаниями. добавляет в трейн и тест столбец с окончаниями. для окончаний, которые в трейне представлены 
#малым количеством слов (<20), создается окончание 'idx0'. оно же присваивается окончаниям, которые не встречались в train

def prepr(X, Z):
    X['ends']=X['word'].apply(lambda x: x[-2:])
    Z['ends']=Z['word'].apply(lambda x: x[-2:])
    
    temp=X.groupby('ends')['Label'].agg([len])
    idx0=temp[temp.len<20].index
    
    def temp1(x):
        if x in idx0: k='idx0'
        else: k=x
        return k
    
    a=X['ends'].map(temp1)
    le=LabelEncoder()
    le.fit(a)
    X['ends']=le.transform(a)
   
    def temp2(x):
        if x in temp[temp.len>=20].index: k=x
        else: k='idx0'
        return k
    
    b=Z['ends'].map(temp2)
    Z['ends']=le.transform(b)
    
    pass

In [9]:
#Признаки из pymorphy2

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [10]:
#часть речи, падеж, число, начальная форма, слова, идущие подряд с одинаковой начальной формой
train['pos']=train['word'].apply(lambda x: morph.parse(x)[0].tag.POS)
train['case']=train['word'].apply(lambda x: morph.parse(x)[0].tag.case)
train['number']=train['word'].apply(lambda x: morph.parse(x)[0].tag.number)
train['norm']=train['word'].apply(lambda x: morph.parse(x)[0].normal_form)
train['agg_norm']=train.groupby('norm')['Word'].transform('count')

In [11]:
test['pos']=test['word'].apply(lambda x: morph.parse(x)[0].tag.POS)
test['case']=test['word'].apply(lambda x: morph.parse(x)[0].tag.case)
test['number']=test['word'].apply(lambda x: morph.parse(x)[0].tag.number)
test['norm']=test['word'].apply(lambda x: morph.parse(x)[0].normal_form)
test['agg_norm']=test.groupby('norm')['Word'].transform('count')

In [12]:
#часть речи, падеж, число для соседей сверху
train['nb_pos']=np.append(np.array(['fff']), train['pos'][:-1].values)
train['nb_case']=np.append(np.array(['fff']), train['case'][:-1].values)
train['nb_number']=np.append(np.array(['fff']), train['number'][:-1].values)

test['nb_pos']=np.append(np.array(['fff']), test['pos'][:-1].values)
test['nb_case']=np.append(np.array(['fff']), test['case'][:-1].values)
test['nb_number']=np.append(np.array(['fff']), test['number'][:-1].values)

In [13]:
#применим ко всему этому LabelEncoder
le=LabelEncoder()

cols_for_le=['pos','case','number']
ads_cols_for_le=['nb_pos','nb_case','nb_number']
les=[]
for i in cols_for_le:
    les.append(LabelEncoder().fit(train[i].append(test[i]).fillna('fff').astype(str)))
    
for n in range(len(cols_for_le)):
    train[cols_for_le[n]]=les[n].transform(train[cols_for_le[n]].fillna('fff').astype(str))
    train[ads_cols_for_le[n]]=les[n].transform(train[ads_cols_for_le[n]].fillna('fff').astype(str))
    test[cols_for_le[n]]=les[n].transform(test[cols_for_le[n]].fillna('fff').astype(str))
    test[ads_cols_for_le[n]]=les[n].transform(test[ads_cols_for_le[n]].fillna('fff').astype(str))

In [14]:
#добавим наташу 
from natasha import NamesExtractor
func = NamesExtractor()
def function_natasha(word):
    return 1 if func(word) else 0

n_train=train['Word'].apply(function_natasha)
n_test=test['Word'].apply(function_natasha)

In [15]:
#добавим pymystem3
import pymystem3
mystem = pymystem3.Mystem()

def name_from_pymystem(word):
    try:
        return 1 if 'имя' in mystem.analyze(word)[0]['analysis'][0]['gr'].split(',') else 0
    except:
        return 0

def surn_from_pymystem(word):
    try:
        return 1 if 'фам' in mystem.analyze(word)[0]['analysis'][0]['gr'].split(',') else 0
    except:
        return 0

train_nm= train['Word'].apply(name_from_pymystem)
train_sur = train['Word'].apply(surn_from_pymystem)
test_nm= test['Word'].apply(name_from_pymystem)
test_sur = test['Word'].apply(surn_from_pymystem)

In [16]:
# и еще пожалуй добавим данные про еще одного соседа сверху
a=np.hstack([np.append(np.array([0,0]), train['is_starts'][:-2].values).reshape(-1, 1),
            np.append(np.array([0]), train['nb_pos'][:-1].values).reshape(-1, 1),
            np.append(np.array([0]), train['nb_case'][:-1].values).reshape(-1, 1),
            np.append(np.array([0]), train['nb_number'][:-1].values).reshape(-1, 1),
             n_train.reshape(-1, 1), np.append(np.array([0]), n_train[:-1].values).reshape(-1, 1),
            train_nm.values.reshape(-1, 1),train_sur.values.reshape(-1, 1),
            np.append(np.array([0]), train_nm[:-1].values).reshape(-1, 1),
            np.append(np.array([0]), train_sur[:-1].values).reshape(-1, 1)])


b=np.hstack([np.append(np.array([0,0]), test['is_starts'][:-2].values).reshape(-1, 1),
   np.append(np.array([0]), test['nb_pos'][:-1].values).reshape(-1, 1),
   np.append(np.array([0]), test['nb_case'][:-1].values).reshape(-1, 1),
   np.append(np.array([0]), test['nb_number'][:-1].values).reshape(-1, 1),
            n_test.reshape(-1, 1), np.append(np.array([0]), n_test[:-1].values).reshape(-1, 1),
            test_nm.reshape(-1, 1),test_sur.reshape(-1, 1),
            np.append(np.array([0]), test_nm[:-1].values).reshape(-1, 1),
            np.append(np.array([0]), test_sur[:-1].values).reshape(-1, 1)])

/home/olga/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/home/olga/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  app.launch_new_instance()
/home/olga/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [17]:
#Формируем файл с ответами

prepr(train, test)

xgb=XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=500)
ohe=OneHotEncoder()
ohe.fit(train.iloc[:,19].reshape(-1, 1))
trn=hstack((train[train.columns[3:14].append(train.columns[15:-1])],ohe.transform(train.iloc[:,19].reshape(-1, 1)),a))
tsn=hstack((test[train.columns[3:14].append(train.columns[15:-1])],ohe.transform(test.iloc[:,18].reshape(-1, 1)),b))
xgb.fit(trn, train.Label)
y=xgb.predict_proba(tsn)[:,1]
#pd.DataFrame(y,columns=['Prediction']).to_csv('subm_2_1', index_label='Id')

/home/olga/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys
/home/olga/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/home/olga/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':


In [14]:
#кривая постобработка в 2 часа ночи) 
tmp=test[['Word', 'apst']]
tmp['f_dbl']=tmp.groupby('Word')['apst'].transform('count')
tmp['y']=y.Prediction
tmp['res']=[999]*tmp.shape[0]
tmp.loc[tmp.apst==1, 'res']=[1]*sum(tmp.apst==1)
for w in tmp[tmp.f_dbl==2]['Word'].unique(): 
    l=np.append(l, tmp[tmp.Word==w].index.values)
tmp.loc[l, 'res']=[0,1]*len(rrr)
tmp['res1']=tmp.apply(lambda x: x[3] if x[4]==999 else x[4], axis=1 )
pd.DataFrame(tmp.res1.values,columns=['Prediction']).to_csv('subm_2_3', index_label='Id')

/home/olga/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
